Code blocks inspired by Sanbomics (GitHub: mousepixels), M. Sanbouw (2023), optimised for current dataset

In [4]:
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd
import numpy as np

In [5]:
alldata = sc.read_h5ad('new_data/annotated/alldata.h5ad')
adata_backup = alldata.X.copy()
alldata = alldata [:, ~alldata.var_names.duplicated(keep='first')].copy()
alldata.obs_names_make_unique()

/net/virus/linuxhome/vivek-group/maria/virtual/lib/python3.12/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [1]:
# !pip install goatools

This notebook performs Gene Ontology (GO) enrichment on genesets from DE analysis. To perform GO enrichment we normally use 
significantly DE genes (as list of gene names), unranked by significance but either upregulated or downregulated, separately. 
GO analysis is an overrepresentation analysis (ORA) which determines whether specific functions are more represented under a certain 
treatment than in an naive (background) set of genes and expected by chance. Gene Ontology is a database which stores terms of biological functions ranging from general (e.g. 'cell cycle') to specific ('auxin signalling in root') and they are linked to genes.
Here we first get these gene-to-go term associations as a file, then get the hierarchy of GO terms (general to specific) as a directed acyclic graph, extract query and background genesets and run enrichment analysis via goatools. 
We also benchmarked goatools for this dataset against the webtool DAVID

In [6]:
import goatools
from goatools.base import download_ncbi_associations
gene2go = download_ncbi_associations() #dowload the gene to GO associations
from goatools.anno.genetogo_reader import Gene2GoReader
geneid2gos_mouse = Gene2GoReader(gene2go, taxids=[10090]) #then get gene to go for mouse with all associations incl non protein coding
from goatools.base import download_go_basic_obo
from goatools.obo_parser import GODag
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS
obo_fname = download_go_basic_obo()
obodag = GODag("go-basic.obo")
geneid2gos = geneid2gos_mouse.get_ns2assc() # returns a dict of dicts with first level keys the main GO classifications the bological process (BP), 
#cellular component (CC) and molecular function (MF) and the second set of keys are the genes whereas the values of every gene are GO terms 
# irrespective of GO term hierarchy (which is instead present in the .obo file)

  EXISTS: gene2go
HMS:0:01:30.719513 612,793 annotations, 29,812 genes, 19,327 GOs, 1 taxids READ: gene2go 
  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2025-02-06) 43,597 Terms


In gene to GO files the associations are with Entrez (e.g. NCBI) gene IDs, thus often it is needed to convert the gene names to
the suitable format through Biomart (pybiomart as Python implementation)

In [7]:
from pybiomart import Server
server = Server(host='http://www.ensembl.org')

mart = server.marts['ENSEMBL_MART_ENSEMBL']

dataset = mart.datasets['mmusculus_gene_ensembl']

results = dataset.query(
    attributes=['entrezgene_id', 'mgi_symbol']
) # this returns a pandas dataframe with the corresponding Entrez gene IDs


In [2]:
# !pip install pybiomart

The background genes are all the genes appearing in the dataset as they are representative of all the genes in the treated and untreated condition yet are not completely unrelated to the dataset state. Also export background genes for input for DAVID. The query genes are already present in files as output of the Differential Expression notebook.

In [8]:
backgr_genes = list(alldata.var['gene_name'].values)
df = pd.DataFrame(backgr_genes, columns=["gene_name"])
df.to_csv("new_data/tables/background_genes.csv", index=False) # export background geneset for DAVID

In [11]:
# geneset = pd.read_csv('go_neurons/go_geneset_high.csv') # gene name column can be fed to DAVID for query geneset
# geneset=list(geneset['gene_name'])
# geneset

Get the corresponding Entrez (NCBI) gene IDs for the gene symbols for input to goatools

In [12]:
def get_assoc_geneset(df, my_genes):
    '''
    Map available gene names to Entrez IDs obtained from Biomart
    Args:
        df (pd.DataFrame): the output of Biomart
        my_genes (list): list of str, the gene names of either the test or the background set
    '''
    filtered = (df[df['MGI symbol'].isin(my_genes)]).dropna()
    new = list(filtered['NCBI gene (formerly Entrezgene) ID'].values)
    new = [int(x) for x in new]
    return new


We obtain our test and background genesets as IDs

In [14]:

geneset = get_assoc_geneset(results, geneset) # converts them to entrez IDs
backgr_genes = get_assoc_geneset(results, backgr_genes) # converts them to entrez IDs

Here we obtain the associations for the background geneset, iterating through the outer keys (the namespaces BP, MF, CC) and the inner keys (all genes with GO associations). These associations are simply the associations as present in the database. This step does not test for enrichment of certain GO terms in the background geneset even if there are GO terms linked to each gene from it.

In [15]:
go_associations={}
for key in geneid2gos.keys():
    first_level_dict = geneid2gos[key]
    go_associations[key]={k: first_level_dict[k] for k in backgr_genes if k in first_level_dict}



In [1]:
# geneid2gos

Here we initialise an object containing the components for GO analysis which goatools will use to evaluate the test geneset, e.g. the DAG, the background geneset and its GO associations. We use default values for multiple test correction method (e.g. False Discovery Rate Benjamin-Hochberg) and the significance threshold of 0.05. propagate_counts=False means that the goatools will identify only the terms directly associated with the genes from the geneset as opposed to these terms + all terms in an upstream position in the hierarchy connected to that term.

In [16]:
#-------------------------------------------------------actual analysis

goeaobj = GOEnrichmentStudyNS(
        backgr_genes, # List of genes in the specific dataset as background
        go_associations, # geneid/GO associations 
        obodag, # Ontologies DAG
        propagate_counts = False,
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh']) # defult multiple-test correction method


Load BP Ontology Enrichment Analysis ...
 84% 12,759 of 15,108 population items found in association

Load CC Ontology Enrichment Analysis ...
 88% 13,235 of 15,108 population items found in association

Load MF Ontology Enrichment Analysis ...
 80% 12,103 of 15,108 population items found in association


We see that a certain GO term is duplicated many times - since it is associated with more than a single gene.

In [20]:
GO_items = []

temp = goeaobj.ns2objgoea['BP'].assoc
for item in temp:
    GO_items += temp[item]
    

temp = goeaobj.ns2objgoea['CC'].assoc
for item in temp:
    GO_items += temp[item]
    

temp = goeaobj.ns2objgoea['MF'].assoc
for item in temp:
    GO_items += temp[item]

In [21]:
GO_items.count('GO:0001525')

246

In [17]:
res = goeaobj.run_study(geneset)


Runing BP Ontology Analysis: current study set of 1768 IDs.
 85%  1,507 of  1,768 study items found in association
100%  1,768 of  1,768 study items found in population(15108)
Calculating 11,991 uncorrected p-values using fisher_scipy_stats
  11,991 terms are associated with 12,753 of 15,108 population items
   5,073 terms are associated with  1,507 of  1,768 study items
  METHOD fdr_bh:
       0 GO terms found significant (< 0.05=alpha) (  0 enriched +   0 purified): statsmodels fdr_bh
       0 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)

Runing CC Ontology Analysis: current study set of 1768 IDs.
 90%  1,587 of  1,768 study items found in association
100%  1,768 of  1,768 study items found in population(15108)
Calculating 1,782 uncorrected p-values using fisher_scipy_stats
   1,782 terms are associated with 13,230 of 15,108 population items
     907 terms are associated with  1,587 of  1,768 study items

In [3]:
# res

In [18]:
print(res[0])

GO:0033629	BP	e	negative regulation of cell adhesion mediated by integrin	6/1768	8/15108	5.78e-05	6	6	0.693	13078, 17829, 19247, 20583, 83964, 245555


In [ ]:
# res = pd.DataFrame(list(map(lambda x: [x.GO, ]

This transforms goatools output into a DataFrame, which contains the names of the terms along with their main classifications (e.g. BP, MF and CC) and also their p-values and adjusted p-values along with the genes that they are associated with

In [26]:
# goea_results_sig = res #[r for r in res if r.p_fdr_bh < 0.05]
GO = pd.DataFrame(list(map(lambda x: [x.GO, x.goterm.name, x.goterm.namespace, x.p_uncorrected, x.p_fdr_bh,\
               x.ratio_in_study[0], x.ratio_in_study[1], GO_items.count(x.GO), x.study_items], res)), columns = ['GO', 'term', 'class', 'p', 'p_corr', 'n_genes',\
                                                'n_study', 'n_go', 'study_genes'])

# GO = GO[GO.n_genes > 1]

In [27]:
GO.to_csv('new_data/tables/go.csv')

,GO,term,class,p,p_corr,n_genes,n_study,n_go,study_genes
0,GO:0033629,negative regulation of cell adhesion mediated ...,biological_process,0.000058,0.69342,6,1768,8,"{17829, 20583, 19247, 245555, 13078, 83964}"
1,GO:0070663,regulation of leukocyte proliferation,biological_process,0.001600,1.00000,3,1768,3,"{15242, 20852, 12798}"
2,GO:0043372,"positive regulation of CD4-positive, alpha-bet...",biological_process,0.001600,1.00000,3,1768,3,"{12522, 22724, 105855}"
3,GO:0003365,establishment of cell polarity involved in ame...,biological_process,0.001600,1.00000,3,1768,3,"{75723, 56332, 27494}"
4,GO:1900195,positive regulation of oocyte maturation,biological_process,0.001600,1.00000,3,1768,3,"{12173, 20878, 64383}"
...,...,...,...,...,...,...,...,...,...
18116,GO:0030626,U12 snRNA binding,molecular_function,1.000000,1.00000,0,1768,1,{}
18117,GO:0042610,CD8 receptor binding,molecular_function,1.000000,1.00000,1,1768,9,{14360}
18118,GO:0160186,phospholipase C inhibitor activity,molecular_function,1.000000,1.00000,0,1768,1,{}
18119,GO:0015203,polyamine transmembrane transporter activity,molecular_function,1.000000,1.00000,0,1768,3,{}
